# Data Wrangling - Part 2

This notebook contains additional data wrangling for my first capstone project.  It is meant to supplement the data wrangling contained in the first data wrangling notebook.  Because of GitHub's limitations on file sizes, I chose to separate the portions of data wrangling that included downloading, storing, and merging data from this portion where I generate additional features.

In [74]:
#Import packages

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [75]:
df = pd.read_csv('C:/Users/robva/Dropbox (Personal)/Springboard Data Science Course/Capstone Project 1/EDA/file_for_analysis.csv')
df = df.drop('Unnamed: 0', axis=1)
df.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg,station_id,...,low_temperature,precipitation,hours_sunlight,avg_temperature_missing,high_temperature_missing,low_temperature_missing,precipitation_missing,hours_sunlight_missing,extreme_value,visitors_recode
0,air_00a91d42b08b08d9,2016-07-01,35,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Friday,0,tokyo__tokyo-kana__tonokyo,...,22.0,0.0,4.4,False,False,False,True,False,74.297411,35.0
1,air_00a91d42b08b08d9,2016-07-02,9,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Saturday,0,tokyo__tokyo-kana__tonokyo,...,23.4,0.0,6.9,False,False,False,False,False,74.297411,9.0
2,air_00a91d42b08b08d9,2016-07-03,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Sunday,0,tokyo__tokyo-kana__tonokyo,...,24.6,0.0,7.6,False,False,False,True,False,74.297411,0.0
3,air_00a91d42b08b08d9,2016-07-04,20,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Monday,0,tokyo__tokyo-kana__tonokyo,...,23.6,1.5,7.1,False,False,False,False,False,74.297411,20.0
4,air_00a91d42b08b08d9,2016-07-05,25,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Tuesday,0,tokyo__tokyo-kana__tonokyo,...,20.2,0.0,0.0,False,False,False,False,False,74.297411,25.0


In [76]:
# Create dummy variables for days of the week and merge

days = pd.get_dummies(df.day_of_week)
df = df.join(days)

In [77]:
#Create continuous and discrete variables for day of the month
df['visit_date']=pd.to_datetime(df['visit_date'])
df['day_of_month']=df.visit_date.dt.day
day_of_month = pd.get_dummies(df.day_of_month, prefix='day_of_month')
df = df.join(day_of_month)

In [78]:
#Create rolling means, medians, and standard deviations by restaurant for each 30 day window.  
#Allow calculations for cells where less than 30 observations are available by setting min_periods = 1.
df['rolling_mean'] = df.groupby('air_store_id')['visitors_recode'].rolling(30, min_periods=1).mean().reset_index(0,drop=True)
df['rolling_median'] = df.groupby('air_store_id')['visitors_recode'].rolling(30, min_periods=1).median().reset_index(0,drop=True)
df['rolling_std'] = df.groupby('air_store_id')['visitors_recode'].rolling(30, min_periods=1).std().reset_index(0,drop=True)
#Std isn't being calculated for about 800 observations so I'm going to fill NaN's with the overall std for each restaurant
df['rolling_std'] = df.groupby(['air_store_id'])['rolling_std'].fillna(df['visitors_recode'].std()).reset_index(0,drop=True)

In [79]:
#Create one week lags for rolling means, medians, and standard deviations and replace NaN's with the mean of each by restaurant
df['rolling_mean_lag'] = df.groupby('air_store_id')['rolling_mean'].shift(7).fillna(df['visitors_recode'].mean()).reset_index(0,drop=True)
df['rolling_median_lag'] = df.groupby('air_store_id')['rolling_median'].shift(7).fillna(df['visitors_recode'].median()).reset_index(0,drop=True)
df['rolling_std_lag'] = df.groupby('air_store_id')['rolling_std'].shift(7).fillna(df['visitors_recode'].std()).reset_index(0,drop=True)

In [80]:
#Create one week lag for each restaurant's visitor count and replace missing with mean number of visitors for that restaurant.
df['visitors_lag'] = df.groupby('air_store_id')['visitors_recode'].shift(7).fillna(df['visitors_recode'].mean()).reset_index(0,drop=True)

In [81]:
#Create dummies for restaurant type
restaurant_type = pd.get_dummies(df.air_genre_name, prefix='genre')
df = df.join(restaurant_type)

In [82]:
#Create dummies for region
region = pd.get_dummies(df.air_area_name, prefix='region')
df = df.join(region)

In [86]:
df.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg,station_id,...,region_Ōsaka-fu Sakai-shi Minamikawaramachi,region_Ōsaka-fu Suita-shi Izumichō,region_Ōsaka-fu Ōsaka-shi Fuminosato,region_Ōsaka-fu Ōsaka-shi Kyōmachibori,region_Ōsaka-fu Ōsaka-shi Kyūtarōmachi,region_Ōsaka-fu Ōsaka-shi Nakanochō,region_Ōsaka-fu Ōsaka-shi Nanbasennichimae,region_Ōsaka-fu Ōsaka-shi Shinmachi,region_Ōsaka-fu Ōsaka-shi Ōgimachi,region_Ōsaka-fu Ōsaka-shi Ōhiraki
0,air_00a91d42b08b08d9,2016-07-01,35,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Friday,0,tokyo__tokyo-kana__tonokyo,...,0,0,0,0,0,0,0,0,0,0
1,air_00a91d42b08b08d9,2016-07-02,9,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Saturday,0,tokyo__tokyo-kana__tonokyo,...,0,0,0,0,0,0,0,0,0,0
2,air_00a91d42b08b08d9,2016-07-03,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Sunday,0,tokyo__tokyo-kana__tonokyo,...,0,0,0,0,0,0,0,0,0,0
3,air_00a91d42b08b08d9,2016-07-04,20,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Monday,0,tokyo__tokyo-kana__tonokyo,...,0,0,0,0,0,0,0,0,0,0
4,air_00a91d42b08b08d9,2016-07-05,25,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Tuesday,0,tokyo__tokyo-kana__tonokyo,...,0,0,0,0,0,0,0,0,0,0


In [106]:
#Create variable for number of restaurants in region - this will proxy for local competition
competition = df.groupby(['visit_date','air_area_name'])['air_store_id'].count().reset_index()
competition.columns = ['visit_date', 'air_area_name', 'competition']
df = df.merge(competition, how = 'left', on = ['visit_date','air_area_name'])

In [107]:
#Export file to EFA
df.to_csv('C:/Users/robva/Dropbox (Personal)/Springboard Data Science Course/Capstone Project 1/EDA/file_for_analysis_1.csv')